In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow.keras
import numpy as np
import os
import json
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import sys

os.chdir("/workspace/telemed5000/ecg/ecg")
import network
import load
from MITDBDataProvider import *
os.chdir('../')
import sys
gpu = sys.argv[-1]

# The cool stuff

In [2]:
def average_weights(models):
    weights = [model.get_weights() for model in models]
    new_weights = list()

    for weights_list_tuple in zip(*weights):
        new_weights.append(
            np.array([np.array(weights_).mean(axis=0)\
                for weights_ in zip(*weights_list_tuple)]))
    return new_weights

In [3]:
params = json.load(open("examples/cinc17/config.json", 'r'))
params.update({
    "input_shape": [None, 1],
    "num_categories": 4
})
def create_model():
    with tf.device('/gpu:'+gpu):
        model = network.build_network(**params)
        optimizer = Adam(
            lr=params["learning_rate"],
            clipnorm=params.get("clipnorm", 1))

        model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
        return model

def fit_model(model, index, dataLoader):
    with tf.device('/gpu:'+gpu):
        train_x, train_y = dataLoader.clientData(index)
        model.fit(train_x, train_y)
        return model

In [4]:
def fit_federated(clients_count, federated_epochs, loader):
    models = list()
    print("Starting to create models...")
    for i in range(clients_count):
        sys.stdout.write("\r\x1b[K"+(i+1).__str__())
        sys.stdout.flush()
        model = create_model()
        models.append(model)
        
    print("Starting the training...")
    
    history = []
    for epoch in range(federated_epochs):
        print(f"Epoch {epoch}/{federated_epochs}")
        for index, model in enumerate(models):
            fit_model(model, index, loader)
        new_weights = average_weights(models)
        for model in models:
            model.set_weights(new_weights)
        score = model.evaluate(loader.validationData[0], loader.validationData[1], verbose=0)
        history.append(score)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
    return model, history

# Data Loader

In [5]:
class CINCFederatedDataLoader:
    
    def __init__(self, train_path, validation_path, clients):
        plain_data = load.load_dataset(train_path)
        self.allData = load.Preproc(*plain_data).process(*plain_data)
        self.clients = clients
        self.groupClientData()
        plain_validation_data = load.load_dataset(validation_path)
        preproc = load.Preproc(*plain_validation_data)
        self.validationData = preproc.process(*plain_validation_data)
    
    def clientData(self, index):
        return self.clientsData[index]
    
    def groupClientData(self):
        self.clientsData = []
        xSplit = self.splitArray(self.allData[0])
        ySplit = self.splitArray(self.allData[1])
        for index in range(self.clients):
            self.clientsData.append([xSplit[index], ySplit[index]])

    def splitArray(self, array):
        return np.array_split(array, self.clients)

In [6]:
def do_oversampling(x, y):
    classes = np.argmax(y,axis=1)
    class_counts=np.bincount(classes)
    max_class = np.max(class_counts)
    oversample = max_class - class_counts
    
    result_y = y
    result_x = x
    idx = [np.random.choice(np.where(classes == c)[0],
                            oversample[c]) for c,c_count in enumerate(class_counts)
                           if c_count > 0 ]
    for i in idx:
        result_y = np.concatenate([result_y, y[i]])
        result_x = np.concatenate([result_x, x[i]])
    
    classes = np.argmax(result_y,axis=1)
    class_counts=np.bincount(classes)
    print(class_counts)
    return result_x, result_y

# Fit cinc17

In [8]:
data_json = "examples/cinc17/train.json"
clients_count = 10
epochs = 20
cincLoader = CINCFederatedDataLoader(data_json, params['dev'], clients_count)
model, history = fit_federated(clients_count=clients_count, federated_epochs=epochs,loader=cincLoader)

100%|██████████| 852/852 [00:00<00:00, 2852.35it/s]


Starting to create models...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
3Starting the training...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1535/1535 [==============================] - 14s 9ms/sample - loss: 0.6636 - acc: 0.7402
Test loss: 1.2958969907581526
Test accuracy: 0.50543875
1535/1535 [==============================] - 9s 6ms/sample - loss: 0.5427 - acc: 0.7740
Test loss: 0.7889342319237794
Test accuracy: 0.74938834
1535/1535 [==============================] - 9s 6ms/sample - loss: 0.4747 - acc: 0.7992
Test loss: 0.49526276219059046
Test accuracy: 0.7996264
1535/1535 [==============================] - 9s 6ms/sample - loss: 0.4153 - acc: 0.8265
Test loss: 0.4161932182983613
Test accuracy: 0.8218442
1535/1535 [==============================] - 9s 6ms/sample - loss: 0.3859 - acc: 0.8341
Test loss: 0.391312580153416
Test accuracy: 0.83128345
1535/1535 [===

In [13]:
import pickle
with open(f"train_cinc_fed_{clients_count}_{epochs}.pkl", "wb") as f:
    pickle.dump(history, f)

NameError: name 'epochs' is not defined

In [ ]:
model.save(f"train_cinc_fed_{clients_count}_{epochs}.h5")